In [51]:
import numpy as np
np.seterr(divide="ignore")

def switch_array_columns(array:np.ndarray, i1: int, i2: int) -> np.ndarray:
    new_array = array.copy()
    match new_array.shape:
        case (_,):
            new_array[[i1, i2]] = new_array[[i2, i1]]

        case (_, _):
            new_array[:,[i1, i2]] = new_array[:, [i2, i1]]

        case _:
            raise(ValueError)
            
    return new_array

def iteracion_simplex(c: np.ndarray, A: np.ndarray, b: np.ndarray, order: np.ndarray | None = None) -> dict:
    if len(c.shape) == 1:
        raise ValueError("C debe ser un vector columna")
    if len(b.shape) == 1:
        raise ValueError("b debe ser un vector columna")
    
    m, n = A.shape
    if order is None:
        print('sube')
        order = np.arange(0, m+n)
    
    # Separar matriz A en variables básicas y no básicas
    B = A.copy()[:,0:m]
    N = A.copy()[:, m:n]
    
    #Costos c básicos y no básicos
    c_B = c.copy()[0:m, 0]
    c_N = c.copy()[m:n, 0]

    try:
        B_inv = np.linalg.inv(B)
    except Exception as _:
        print('singular matrix')
        i1 = np.random.randint(0, n)
        i2 = np.random.randint(0, n)
        A_new = switch_array_columns(A, i1, i2)
        new_order = switch_array_columns(order, i1, i2)
        return iteracion_simplex(c, A_new, b, new_order)
    
    # solución inicial z0 = z(x0)
    x0 = B_inv@b
    z0 = (c_B.T)@(B_inv@b)

    if any(x < 0 for x in x0):
        print('non basic solution')
        i1 = np.random.randint(0, n)
        i2 = np.random.randint(0, n)
        A_new = switch_array_columns(A, i1, i2)
        new_order = switch_array_columns(order, i1, i2)
        return iteracion_simplex(c, A_new, b, new_order)

    pi = c_B.T@B_inv
    costos_reducidos = pi@N - c_N.T
    b_barra = B_inv@b
    Y = B_inv@N

    return {
        "z": z0,
        "x": x0,
        "pi": pi,
        "c_reducidos": costos_reducidos,
        "b_barra": b_barra,
        "Y": Y,
        "order": order
    }



In [52]:
def metodo_simplex_revisado(c: np.ndarray, A: np.ndarray, b: np.ndarray, order: np.ndarray | None = None):
    # TODO implementar el orden para distinguir las variables y ver que putas cambia el resultado con distintos args
    
    m, n = A.shape
    inicial = iteracion_simplex(c, A, b)
    c_reducidos = inicial['c_reducidos']

    if all(x <= 0 for x in c_reducidos):
        return inicial
    indice_no_basicas = np.argmax(c_reducidos)
    
    Y = inicial['Y']
    b_barra = inicial['b_barra']
    cocientes = np.divide(b_barra, np.vstack(Y[:, indice_no_basicas]))

    indice_basicas = np.where(cocientes > 0, cocientes, np.inf).argmin()

    i1, i2 = indice_basicas, indice_no_basicas + m

    A_new = switch_array_columns(A, i1, i2)
    c_new = switch_array_columns(c.T, i1, i2).T
    order_new = switch_array_columns(inicial['order'], i1, i2)

    return metodo_simplex_revisado(c_new, A_new, b, order_new)

### Demo

In [53]:
A = np.array([
    [1,0,0,1,0],
    [0,1,0,0,1],
    [0,0,1,3,2]
])
c = np.vstack([0,0,0,-3,-5])
b = np.vstack([4, 6, 18])

metodo_simplex_revisado(c,A,b)

sube
sube
sube


{'z': array([-36.]),
 'x': array([[2.],
        [6.],
        [2.]]),
 'pi': array([ 0., -3., -1.]),
 'c_reducidos': array([-1., -3.]),
 'b_barra': array([[2.],
        [6.],
        [2.]]),
 'Y': array([[-0.33333333,  0.66666667],
        [ 0.        ,  1.        ],
        [ 0.33333333, -0.66666667]]),
 'order': array([0, 1, 2, 3, 4, 5, 6, 7])}

Con otro orden se comporta raro

In [63]:
A = np.array([[3, 2, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]])
c = np.vstack([-3,-5,0,0,0])
b = np.vstack([18, 4, 6])
metodo_simplex_revisado(c,A,b)


sube
non basic solution
non basic solution
singular matrix


{'z': array([-38.]),
 'x': array([[6.],
        [4.],
        [6.]]),
 'pi': array([ 0., -5., -3.]),
 'c_reducidos': array([-3., -5.]),
 'b_barra': array([[6.],
        [4.],
        [6.]]),
 'Y': array([[ 1.,  0.],
        [ 0.,  1.],
        [ 2., -3.]]),
 'order': array([4, 0, 2, 1, 3, 5, 6, 7])}

# Ejercicio de la Tarea

Demanda:
$$ P_1 + P_2 + P_3 = 475 $$

Límites superiores de operación:
$$ P_1 + s_1 = 175 $$
$$ P_2 + s_2 = 300 $$
$$ P_3 + s_3 = 150 $$

Límites inferiores:
$$ P_3 - s_4 =50 $$

Restricción de las lineas:
$$ P_1 + s_5 = 160 $$
$$-P_3 + s_6 = -25 $$

$$\begin{bmatrix}
    1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
    1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\ 
    0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 & 0 & 0 &-1 & 0 & 0 \\
    1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
    0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 &-1 
\end{bmatrix}
 = \begin{bmatrix}
    475 \\ 175 \\ 300 \\ 150 \\ 50 \\ 160 \\ 25
\end{bmatrix}
$$


In [55]:
from scipy.optimize import linprog

c = np.array([50,60,75])
A_eq = [[1, 1, 1]]
b_eq = [475]

bounds = [(0,160), (0,300), (50,150)]
res = linprog(c=c, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
res

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 27650.0
              x: [ 1.600e+02  2.650e+02  5.000e+01]
            nit: 0
          lower:  residual: [ 1.600e+02  2.650e+02  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  1.500e+01]
          upper:  residual: [ 0.000e+00  3.500e+01  1.000e+02]
                 marginals: [-1.000e+01  0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00]
                 marginals: [ 6.000e+01]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [73]:
c = np.vstack([50, 60, 75, 0, 0, 0, 0, 0, 0])
A = np.array([
    [1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 0,-1, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, -1, 0, 0, 0, 0, 0, 1],
])
b = np.vstack([475, 175, 300, 150, 50, 160, 25])

metodo_simplex_revisado(c, A, b)

sube
non basic solution
sube


{'z': array([27650.]),
 'x': array([[160.],
        [265.],
        [ 50.],
        [ 15.],
        [ 35.],
        [100.],
        [ 75.]]),
 'pi': array([ 60.,   0.,   0.,   0.,  15., -10.,   0.]),
 'c_reducidos': array([-10., -15.]),
 'b_barra': array([[160.],
        [265.],
        [ 50.],
        [ 15.],
        [ 35.],
        [100.],
        [ 75.]]),
 'Y': array([[ 1.,  0.],
        [-1.,  1.],
        [ 0., -1.],
        [-1.,  0.],
        [ 1., -1.],
        [ 0.,  1.],
        [ 0., -1.]]),
 'order': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])}